## Basic PN junction with no applied bais

Consider a step junction in which the doping concentration is uniform in each region and there is an abrupt change in doping at the junc- tion. Initially, at the metallurgical junction, there is a very large density gradient in both electron and hole concentrations.

Majority carrier electrons in the n region will begin diffusing into the p region, and majority carrier holes in the p region will begin diffusing into the n region.

Based on two assumptions:
1. Boltzmann approximation is valid, which means that each semiconductor region is nondegenerately doped.
2. Complete ionization exists, which means that the tempera- ture of the pn junction is not “too low.”

### Bandgap diagram
![Alt text](./images/1.png)

### Built-in Potential Barrier 
![Alt text](./images/2.png)

The electric field is determined from Poisson’s equation, which, for a one- dimensional analysis, is
![Alt text](./images/3.png)
![Alt text](./images/11.png)
### Space Charge Width
![Alt text](./images/6.png)
![Alt text](./images/4.png)
![Alt text](./images/5.png)

### Built-in Electric Field
![Alt text](./images/9.png)
![Alt text](./images/10.png)

### Consider Temperature
![Alt text](./images/12.png)
![Alt text](./images/13.png)
![Alt text](./images/14.png)

### Temperature dependency of Eg of Si
![Alt text](./images/15.png)
![Alt text](./images/16.png)

assign values for releveant parameters

In [1]:
from sympy import Symbol
import math
from decimal import Decimal
import numpy as np
# some constants needed
# m0 = 9.11 x 10^-31 kg is the free electron rest mass.
H = 6.62607004 * 10**(-34) # plank constant whose unit is m2kg/s
M0 = 9.11 * 10**(-31) # kg
EG_0 = 1.166 # ev
alpha = 4.73 * 10**(-4) #ev/k
beta = 636 # k 
# 300K At 300 K (room temperature), kT = 0.0259 eV.
RT = 300 # room temperature
KT_ev = 0.0259
VT = 0.02586 # vt = kt/e thermal voltage unit: v
N_V = 1.04 * 10**19
N_C = 2.8 * 10**19
Na_magnitude = 10**16
Nd_magnitude = 10**15
permittivity_si = 11.7 * (8.85 * 10**(-14)) # unit is F/cm^2
e = 1.6 * 10**(-19)
# kT is the product of the Boltzmann constant and the temperature,
# calculate the intrinsic electron concentration's square

In [2]:
def cal_ni_square(temperature=300):
    if temperature == 300:
        ni_square = (1.5 * 10**10)**2
    else:
        ratio = temperature/RT
        change_term = ratio**(3/2)
        Nv = N_V * change_term
        Nc = N_C * change_term
        band_gap = cal_bandgap(temperature)
        kt = KT_ev * ratio
        ni_square = Nv * Nc * math.exp(-band_gap/kt)
#     ni_square = math.sqrt(ni_square)
    print('When T is {}k, ni^2 is {:2E}cm^(-6)'.format(temperature,ni_square))
    return ni_square

In [3]:
# the unit of temperature parameter is k
def cal_bandgap(temperature=300):
    Eg = EG_0 - (alpha * temperature**2)/(temperature + beta)
    Eg = round(Eg,3)
    print('When T is {}k, the bandgap of Si is {}ev.'.format(temperature,Eg))
    return Eg

In [4]:
def na_nd(na=1, nd=1):
    return Na_magnitude * na, Nd_magnitude * nd

In [15]:
def cal_vbi(na=1, nd=1, t=300):
    Na, Nd = na_nd(na=na, nd=nd)
    ni_square = cal_ni_square(temperature=t)
#     ni_square = (1.5 * 10**10)**2
    vbi = VT * math.log(Na * Nd / ni_square)
    return vbi  # unit is v

In [11]:
# space charge width calculation
# permittivity of the semiconductor
def cal_space_charge_width(na=1, nd=1, t=300, applied_voltage=0):
    Na, Nd = na_nd(na=na, nd=nd)
    Na_Nd = Na / Nd
    Nd_Na = 1/Na_Nd
    Na_plus_Nd = Na + Nd
    vbi = cal_vbi(na=na,nd=nd,t=t)
    # the default value of applied_voltage is zero, reverse bais is positive
    first_part = (2*permittivity_si*(vbi+applied_voltage))/e
    third_part = 1/Na_plus_Nd
    # vectoralization
    xn = (first_part * Na_Nd * third_part)**(1/2)
    xp = (first_part * Nd_Na * third_part)**(1/2)
    w = xn + xp
    width = np.array([xn,xp,w])
    # the units of xn xp and w are all cm
    # convert the unit from cm to μm time 10 to the power 4/ 10 to the 4
    return np.around(width*10**4,decimals=4)

In [16]:
# built in max electric field without applied bais
def cal_max_electric_field(na=1, nd=1, t=300):
    Na, Nd = na_nd(na=na, nd=nd)
    charge_width = list(cal_space_charge_width(na=na, nd=nd, t=t))
    e_max = (-e * Na)/permittivity_si * charge_width[1] * 10**(-4) # using xp to calculate
    return e_max # unit is v/cm

input the max e field, give the na and nd

In [25]:
def cal_max_electric_field_with_bais(na=1, nd=1, t=300, applied_voltage=0):
    Na, Nd = na_nd(na=na, nd=nd)
    charge_width = list(cal_space_charge_width(na=na, nd=nd, t=t, applied_voltage=applied_voltage))
    vbi = cal_vbi(na=na,nd=nd,t=t)
    e_max = (-2*(vbi+applied_voltage)/(charge_width[2]*10**(-4)))
    return e_max

In [26]:
cal_max_electric_field_with_bais(na=20, nd=8.43, t=300, applied_voltage=25)

When T is 300k, ni^2 is 2.250000E+20cm^(-6)
When T is 300k, ni^2 is 2.250000E+20cm^(-6)


-253795.81976143917

In [9]:
cal_space_charge_width(na=0.5, nd=50, applied_voltage=4)

When T is 300k, ni^2 is 2.250000E+20cm^(-6)


array([0.1054, 1.0536, 1.159 ])

In [27]:
cal_builtin_electric_field(na=1, nd=1, t=300)

When T is 300k, ni^2 is 2.250000E+20cm^(-6)


-13350.717079530641

In [7]:
cal_bandgap(300)

When T is 300k, the bandgap of Si is 1.121ev.


1.121

In [31]:
cal_ni_square()

When T is 300k, the bandgap of Si is 1.121ev.
When T is 300k, ni^2 is 4.646451E+19cm^(-6)


4.646450595667298e+19

In [14]:
cal_vbi(na=0.5, nd=50) # ni is 1.5 * 10**10  experimental result

When T is 300k, ni^2 is 2.250000E+20cm^(-6)


0.7172628289928237

In [49]:
cal_vbi(na=20) # ni is theoretical result

When T is 300k, the bandgap of Si is 1.121ev.
When T is 300k, ni^2 is 4.646451E+19cm^(-6)


0.7522842031586012

### The difference between Vbi calculated based on experimental ni and theoretical ni is 0.04v, is it acceptable? If I wanna use experimental resule, how can I calculate ni at any temperature which is not RM?